In [1]:
%load_ext memory_profiler

from autograd.differential_operators import binomial_checkpoint
import autograd.numpy as np
from autograd import grad

from time import time

seq_len = 1000

param_shape = (300,)
state_shape = (300,)
x_shape = (400,)

inputs = [np.random.randn(*x_shape) for _ in range(seq_len)]
initial_state = np.random.randn(*state_shape)
parameters = np.ones(param_shape)

In [2]:
def simple_rnn(param, state, x):
    assert(param.shape == param_shape)
    assert(state.shape == state_shape)
    assert(x.shape == x_shape)
    
    return param * (state / np.sum(x))

loop, unwrapped_loop = binomial_checkpoint(simple_rnn, seq_len, 100)

In [4]:
f = lambda parameters: loop(parameters, initial_state, inputs)
g = lambda parameters: np.sum(sum(f(parameters)))

start = time()
%memit g1 = (grad(g)(parameters))
end = time()

print("elapsed time: ", end - start, "\n")

f = lambda parameters: unwrapped_loop(parameters, initial_state, inputs)
g = lambda parameters: np.sum(sum(f(parameters)))

start = time()
%memit g2 = (grad(g)(parameters))
end = time()

print("elapsed time: ", end - start, "\n")

print("verify gradient correctness: ", np.sum((g1 - g2)**2))

peak memory: 99.20 MiB, increment: 0.03 MiB
elapsed time:  3.0558979511260986 

peak memory: 108.52 MiB, increment: 9.32 MiB
elapsed time:  4.474081039428711 

verify gradient correctness:  1.1059625669273633e-31
